In [13]:
import os
from html.parser import HTMLParser
from bs4 import BeautifulSoup

In [2]:
folder = "documents"

In [3]:
class CustomHTMLParser(HTMLParser):

    def __init__(self, *, convert_charrefs: bool = ...) -> None:
        super().__init__(convert_charrefs=convert_charrefs)
        self.content : str = ''

    def handle_data(self, data: str) -> str:
        self.content += data
        self.content += '\n'

In [4]:
stopwords = list()
with open("Urdu stopwords.txt", encoding="utf-8") as f:
    stopwords.extend(f.read().split("\n"))

print(stopwords)

['اب', 'ابھی', 'اپنا', 'اپنے', 'اپنی', 'اٹھا', 'اس', 'اسے', 'اسی', 'اگر', 'ان', 'انہوں', 'انہی', 'انہیں', 'انھیں', 'او', 'اور', 'اے', 'ایسا', 'ایسے', 'ایسی', 'ایک', 'آ', 'آپ', 'آتا', 'آتے', 'آتی', 'آگے', 'آنا', 'آنے', 'آنی', 'آئے', 'آئی', 'آئیں', 'آیا', 'با', 'بڑا', 'بڑے', 'بڑی', 'بعد', 'بعض', 'بلکہ', 'بہت', 'بھی', 'بے', 'پاس', 'پر', 'پہلے', 'پھر', 'تا', 'تاکہ', 'تب', 'تجھ', 'تجھے', 'تک', 'تم', 'تمام', 'تمہارا', 'تمہارے', 'تمھارے', 'تمہاری', 'تمہیں', 'تمھیں', 'تھا', 'تھے', 'تھی', 'تھیں', 'تو', 'تیری', 'تیرے', 'جا', 'جاتا', 'جاتی', 'جاتے', 'جاتی', 'جانے', 'جانی', 'جاؤ', 'جائے', 'جائیں', 'جب', 'جس', 'جن', 'جنہوں', 'جنہیں', 'جو', 'جیسا', 'جیسے', 'جیسی', 'جیسوں', 'چاہیئے', 'چلا', 'چاہے', 'چونکہ', 'حالاں', 'حالانکہ', 'دو', 'دونوں', 'دوں', 'دے', 'دی', 'دیا', 'دیں', 'دیے', 'دیتا', 'دیتے', 'دیتی', 'دینا', 'دینے', 'دینی', 'دیئے', 'ڈالا', 'ڈالنا', 'ڈالنے', 'ڈالنی', 'ڈالے', 'ڈالی', 'ذرا', 'رکھا', 'رکھتا', 'رکھتے', 'رکھتی', 'رکھنا', 'رکھنے', 'رکھنی', 'رکھے', 'رکھی', 'رہ', 'رہا', 'رہتا', 'رہتے', 'ر

In [5]:
output_dir = "output_files"
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

In [7]:
docids = dict()
termids = dict()
doc_index = dict()

doc_enum = 1
term_enum = 1

In [24]:
for doc in os.listdir(folder):
    docname = doc.rsplit(".", 1)[0]
    try:
        docids[docname]
    except KeyError:
        docids[docname] = doc_enum
        doc_enum += 1
    
    with open(os.path.join(folder, doc), encoding="utf-8") as f:
        content = f.read()
        parser = CustomHTMLParser()
        parser.feed(content)
        if (bool(BeautifulSoup(parser.content, "html.parser").find())):
            print(doc)
        content = parser.content
        content = content.replace("\n", " ")
        # Removing all english characters.
        content = ''.join([l for l in content if not ((l >= 'a' and l <= 'z') or (l >= 'A' and l <= 'Z'))])
        tokens = content.split(" ")
        for i, tok in enumerate(tokens):
            if tok in stopwords:
                continue
            try:
                termids[tok]
            except KeyError:
                termids[tok] = term_enum
                term_enum += 1
            
            doc_index.setdefault(docids[docname], dict())
            doc_index[docids[docname]].setdefault(termids[tok], list())
            doc_index[docids[docname]][termids[tok]].append(i)


In [31]:
with open(os.path.join(output_dir, "docids.txt"), "w") as f:
    for doc, id in docids.items():
        f.write(f"{id}\t{doc}\n")

with open(os.path.join(output_dir, "termids.txt"), "w") as f:
    for term, id in termids.items():
        f.write(f"{id}\t{term}\n")

with open(os.path.join(output_dir, "doc_index.txt"), "w") as f:
    for docid, val in doc_index.items():
        for termid, postings in val.items():
            f.write("{docid}\t{termid}\t{postings}\n".format(**{"docid" : docid, "termid" : termid, "postings" : '\t'.join([str(p) for p in postings])}))
